In [1]:
#read data

import pandas as pd



data=pd.read_csv("data/macparland/GSE115469_Data.csv",header=0,index_col=0,sep=",")

data=data.T   #transpose to cell by gene

annotation=pd.read_csv("data/macparland/GSE115469_CellClusterType.txt",sep="\t")
annotation["celltype"]=annotation["CellType"]

data.head()

,RP11-34P13.7,FO538757.2,AP006222.2,RP4-669L17.10,RP5-857K21.4,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,RP11-54O7.3,...,AC145212.2,AC011043.1,AL592183.1,AC007325.4,AC007325.2,AL354822.1,AC004556.1,AC233755.2,AC233755.1,AC240274.1
P1TLH_AAACCTGAGCAGCCTC_1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1TLH_AAACCTGTCCTCATTA_1,0.0,0.0,0.31476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1TLH_AAACCTGTCTAAGCCA_1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1TLH_AAACGGGAGTAGGCCA_1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1TLH_AAACGGGGTTCGGGCT_1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
base_dir="data/macparland"

In [3]:
data_sparse=data


#statistics of cells expressing each gene
gene_expressed_cell_number=data_sparse.astype(bool).sum(axis=0)

print(len(gene_expressed_cell_number))
#filter gene expressed in less than 10 cells
gene_expressed_cell_number=gene_expressed_cell_number[gene_expressed_cell_number>500]
print(len(gene_expressed_cell_number))

data_rm_sparse=data_sparse[gene_expressed_cell_number.index.tolist()]
data_rm_sparse.shape           #10k cells, 4487 genes



20007
5946


(8444, 5946)

In [4]:
import pickle
TF_gene_dict=pickle.load(open("human/TF_gene_dict","rb"))

len(TF_gene_dict)

1209

In [5]:
#generate gene_to_TF_transform_matrix
import numpy as np
import pickle

TF_gene_dict=pickle.load(open("human/TF_gene_dict","rb"))


gene_number=len(data_rm_sparse.columns.to_list())    

TF_number=len(TF_gene_dict)

gene_to_TF_transform_matrix=np.zeros((gene_number,TF_number))

TF_list=TF_gene_dict.keys()
for i,gene in enumerate(data_rm_sparse.columns):
    try:
        j=TF_list.index("gene")
        gene_to_TF_transform_matrix[i][j]=1
    except:
        pass
        
gene_to_TF_transform_matrix

pickle.dump(gene_to_TF_transform_matrix,open("%s/gene_to_TF_transform_matrix" %base_dir,"wb"))

In [6]:
#generate TF_mask

import numpy as np

gene_TF_dict=pickle.load(open("human/gene_TF_dict","rb"))

gene_number = len(data_rm_sparse.columns.to_list())    #6033
TF_number = len(TF_gene_dict)

TF_mask = np.zeros((gene_number,TF_number))
error_count=0

for i,gene_id in enumerate(data_rm_sparse.columns):

    for j,TF in enumerate(TF_gene_dict):
        if TF in gene_TF_dict.get(gene_id,[]):
            TF_mask[i][j]=1
        else:
            error_count+=1
        
print(error_count)
print(TF_mask)

pickle.dump(TF_mask,open("%s/TF_mask" %base_dir,"wb"))

4787003
[[0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
#generate GO_mask

GO_dict={}
with open("human/goa_human.gaf") as f:
    for line in f:
        if line[0] == "!":
            continue
        
        gene_id=line.split("\t")[2]
        GO_term=line.split("\t")[4]
        if GO_term not in GO_dict:
            GO_dict[GO_term]=[]
        GO_dict[GO_term].append(gene_id)


GO_list=[]
count=0
for item in GO_dict:
    if len(GO_dict[item])>=30:
        count+=1
        GO_list.append(item)
print(count)


gene_dict={}
with open("human/goa_human.gaf") as f:
    for line in f:
        if line[0]=="!":
            continue
        gene_id=line.split("\t")[2].upper()
        GO_term=line.split("\t")[4]
        if gene_id not in gene_dict:
            gene_dict[gene_id]=[]
        gene_dict[gene_id].append(GO_term)



gene_number=len(gene_expressed_cell_number.index.tolist())    #6033
GO_number=len(GO_list)  

GO_mask=np.zeros((gene_number,GO_number))
error_count=0

for i,gene_id in enumerate(data_rm_sparse.columns):

    for j,GO_term in enumerate(GO_list):
        if GO_term in gene_dict.get(gene_id,"GO:default"):

            GO_mask[i][j]=1
        else:
            error_count+=1
        
print(error_count)

pickle.dump(GO_mask,open("%s/GO_mask" %base_dir,"wb"))


1946
11494620


In [8]:
#generate GO_TF_mask

TF_number=len(TF_gene_dict)
GO_number=len(GO_list) 

GO_TF_mask=np.zeros((TF_number,GO_number))
error_count=0

for i,TF in enumerate(TF_gene_dict):
    for j,GO in enumerate(GO_list):
        if GO in gene_dict.get(TF,"GO:default"):
            GO_TF_mask[i][j]=1
        else:
            error_count+=1
print(error_count)
        
GO_TF_mask

pickle.dump(GO_TF_mask,open("%s/GO_TF_mask" %base_dir,"wb"))


2332169


In [9]:
#GO_Net
import math
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.model_selection import train_test_split as ts

data_rm_sparse=data_rm_sparse

data_rm_sparse.index=annotation["celltype"].to_list()

###############################################################
gene_to_TF_transform_matrix=pickle.load(open("%s/gene_to_TF_transform_matrix" %base_dir,"rb"))
TF_mask=pickle.load(open("%s/TF_mask" %base_dir,"rb"))
GO_mask=pickle.load(open("%s/GO_mask" %base_dir,"rb"))
GO_TF_mask=pickle.load(open("%s/GO_TF_mask" %base_dir,"rb"))
###############################################################
#data_annotation = pd.read_csv('data/macparland/GSE115469_CellClusterType.txt', sep="\t")
#index_rename_dict = {key: value for key, value in zip(data_annotation['CellName'], data_annotation['CellType'])}
#$data_rm_sparse=data_rm_sparse.rename(index=index_rename_dict)

#normalize by row
#data_rm_sparse = data_rm_sparse.apply(lambda row: row / np.linalg.norm(row), axis=1)

#merge similar cell types
#data_rm_sparse.index = data_rm_sparse.index.str.replace('Hepatocyte_\d+', 'Hepatocyte', regex=True)
#data_rm_sparse.index = data_rm_sparse.index.str.replace('gamma-delta_T_Cells_\d+', 'gamma-delta_T_Cells', regex=True)


#filter low count cells
#data_rm_sparse = data_rm_sparse[data_rm_sparse.index != 'Hepatic_Stellate_Cells']


#novel_cell_type = ['Plasma_Cells']

#data_rm_sparse_novel = data_rm_sparse[data_rm_sparse.index.isin(novel_cell_type)]
#data_rm_sparse_rest = data_rm_sparse[~data_rm_sparse.index.isin(novel_cell_type)]




classes=[]
for celltype in data_rm_sparse.index:
    if celltype not in classes:
        classes.append(celltype)
#print(len(classes),classes)


label_dict_revese={}
label_dict={}
for i,celltype in enumerate(classes):
    label_dict[celltype]=i
    label_dict_revese[i]=celltype
label_dict
################################################################



def gen_mask(row,col,percent=0.5,num_zeros=None):
    if num_zeros is None:
        #Total number being masked is 0.5 by default
        num_zeros=int((row*col)*percent)
    
    mask=np.hstack([np.zeros(num_zeros),np.ones(row*col-num_zeros)])
    np.random.shuffle(mask)
    return mask.reshape(row,col)

class LinearFunction(torch.autograd.Function):
    """
    autograd function which masks it's weights by 'mask'.
    """
    
    #Not that both forward and backword are @staticmethod

    
    #bias, mask is an optional argument
    @staticmethod
    def forward(ctx, input, weight, bias=None, mask=None):
        if mask is not None:
            #change weight to 0 where mask == 0

            weight=weight*mask
 
        output=input.mm(weight.t())

        if bias is not None:
            output+=bias.unsqueeze(0).expand_as(output)
        
        ctx.save_for_backward(input, weight, bias, mask)

        return output
    
    #This function has noly a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx,grad_output):
        input,weight,bias,mask = ctx.saved_tensors
        grad_input=grad_weight=grad_bias=grad_mask=None
        
        #These meeds_input_grad checks are optional and there only to improve efficiency.
        #If you want to make your code simpler, you can skip them. Returning gradients for
        #inputs that don't require it is not an error.
        if ctx.needs_input_grad[0]:
            grad_input=grad_output.mm(weight)
        
        if ctx.needs_input_grad[1]:
            grad_weight=grad_output.t().mm(input)
            
            if mask is not None:
                
                #change grad_weight to 0 where mask == 0
                grad_weight=grad_weight*mask

        
        #if bias is not None and ctx.need_input_grad[2]:
        if ctx.needs_input_grad[2]:
            grad_bias=grad_output.sum(0).squeeze(0)
        
        return grad_input,grad_weight,grad_bias,grad_mask
    

       
class CustomizedLinear(nn.Module):
    def __init__(self,input_features,output_features, bias=None, mask=None):
        """
        Arguments
        mask [numpy array]:
            The shape is (n_input_fearues,n_output_features).
            The elements are 0 or 1 which delcare un-connected or connected.
            
        bias [bool]:
            flg of bias.
        """
        super(CustomizedLinear,self).__init__()
        self.input_features=input_features
        self.out_features=output_features
        
        #nn.Parameter is a spetial kind of Tensor, that will get
        #automatically registered as Module's parameter once it's assigned
        #as an attribute
        self.weight=nn.Parameter(torch.Tensor(self.out_features,self.input_features))
        
        if bias:

            self.bias=nn.Parameter(torch.Tensor(self.out_features))
        else:
            #You should always register all possible parameters, but the
            #optinal ones can be None if you want.
            self.register_parameter("bias",None)
            
        #Initialize the above parameters (weight and bias). Important!
        self.init_params()
        
        #mask should be registered after weight and bias
        if mask is not None:
            mask=torch.tensor(mask,dtype=torch.float).t()
            self.mask=nn.Parameter(mask,requires_grad=False)
        else:
            self.register_parameter("mask",None)

        
    def init_params(self):
        stdv=1./math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv,stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv,stdv)
                
    def forward(self,input):
        #See the autograd section for explanation of what happens here.
        
        output=LinearFunction.apply(input,self.weight,self.bias,self.mask)
        
        return output
        
    def extra_repr(self):
        #(Optional) Set the extra informatioin about this module. You can test
        #it by printing an object of this class.
        return "input_features={}, output_features={}, bias={}, mask={}".format(
            self.input_features, self.out_features,
            self.bias is not None, self.mask is not None)
        
        
class GO_Net(nn.Module):
    def __init__(self,in_size,out_size,ratio=[0.006525,0,0]):
        super(GO_Net,self).__init__()

        self.gene_number=len(gene_expressed_cell_number.index.tolist())    #6033
        self.TF_number=1209
        self.GO_number=len(GO_list)
        self.class_number=3

        self.gene_to_TF_transform_matrix=torch.tensor(gene_to_TF_transform_matrix,dtype=torch.float32)
    
        
        self.bn0=nn.BatchNorm1d(self.gene_number)
        #self.fc1=CustomizedLinear(in_size,2290,mask=gen_mask(3443,2290,ratio[0]))  
        #self.fc1=CustomizedLinear(in_size,1946,mask=gen_mask(2903,1946,ratio[0]))        
        self.fc1=CustomizedLinear(in_size,self.GO_number,mask=GO_mask)    #GO_term
        self.gene_to_GO_layer=CustomizedLinear(in_size,self.GO_number,mask=GO_mask)    #GO_term
        #self.fc1=CustomizedLinear(in_size,2290,mask=np.ones((3443,2290)))
    
        self.bn1=nn.BatchNorm1d(self.GO_number)
                
        self.fc2=CustomizedLinear(self.GO_number,out_size,mask=gen_mask(self.GO_number,out_size,ratio[1]))
        self.bn2=nn.BatchNorm1d(out_size)

        self.gene_to_TF_layer=CustomizedLinear(self.gene_number,self.TF_number,mask=TF_mask)
        self.TF_to_GO_layer=CustomizedLinear(self.TF_number,self.GO_number,mask=GO_TF_mask)
        
        self.fc3=CustomizedLinear(100,100,mask=gen_mask(100,100,ratio[1]))

        self.fc4=CustomizedLinear(100,out_size,mask=gen_mask(100,out_size,ratio[1]))
        
        self.relu=nn.ReLU()
        self.leaky_relu=nn.LeakyReLU()
        #self.dropout = nn.Dropout(0.1)
        for module in self.modules():
            if isinstance(module,nn.Linear):
                nn.init.uniform_(module.weight,a=0,b=1)
            elif isinstance(module,(nn.BatchNorm1d,nn.GroupNorm)):
                nn.init.constant_(module.weight,1)
                nn.init.constant_(module.bias,0)

                        
    def forward(self,x):

        #x=self.bn0(x)
        TF_residul=torch.matmul(x,self.gene_to_TF_transform_matrix)

        TF_derived_from_gene=self.gene_to_TF_layer(x)

        TF_sum=TF_residul+TF_derived_from_gene
        #TF_sum=TF_derived_from_gene

        GO_derived_from_TF=self.TF_to_GO_layer(TF_sum)

        GO_derived_from_gene=self.gene_to_GO_layer(x)

        GO_sum=GO_derived_from_TF+GO_derived_from_gene

        #x=self.bn0(x)
        #x=self.fc1(x)
        #x=self.bn1(x)
        #x=self.relu(x)
        #x=self.dropout(x)
        GO_sum=self.leaky_relu(GO_sum)

        #x=torch.tanh(x) 
        #print(161,self.fc1.weight)
        x=self.fc2(GO_sum)
        #x=self.bn2(x)
        #x=self.relu(x)
        #x=self.leaky_relu(x)
        #x=self.fc3(x)
        #x=self.leaky_relu(x)
        #x=self.fc4(x)
 
        return x,GO_sum,TF_derived_from_gene,GO_derived_from_TF

"""
class Reconstraction(nn.Module):
    def __init__(self,in_size,out_size):
        super(Reconstraction,self).__init__()

        self.decoder = nn.Sequential(
            nn.Linear(in_size, 500),
            nn.LeakyReLU(True),
            nn.Linear(500, 500),
            nn.LeakyReLU(True),
            nn.Linear(500, 500),
            nn.LeakyReLU(True),
            nn.Linear(500, 1000),
            nn.LeakyReLU(True),
            nn.Linear(1000, out_size)
        )

    def forward(self, x):
        x = self.decoder(x)
        return x
"""     


 
class MyDataset(Dataset):
    def __init__(self, x,y):
        self.x=x
        self.y=y

    def __getitem__(self, index):
        features = self.x[index]
        label = self.y[index]
        return features, label

    def __len__(self):
        return len(self.x)

def accuracy_score(y_test,y_pred):
    t=0
    f=0
    for i in range(len(y_test)):
        if y_test[i]==y_pred[i]:
            t+=1
        else:
            f+=1
    return(t/(t+f))



from torch.utils.data import WeightedRandomSampler

def make_weights_for_balanced_classes(dataset, nclasses):
    count = [0] * nclasses
    for item in dataset:
        count[item[1]] += 1
    weight_per_class = [0.] * nclasses
    N = float(sum(count))
    for i in range(nclasses):
        weight_per_class[i] = N/float(count[i])
    weight = [0] * len(dataset)
    for idx, val in enumerate(dataset):
        weight[idx] = weight_per_class[val[1]]
    return weight


class CustomWeightedRandomSampler(WeightedRandomSampler):
    """WeightedRandomSampler except allows for more than 2^24 samples to be sampled"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __iter__(self):
        rand_tensor = np.random.choice(range(0, len(self.weights)),
                                       size=self.num_samples,
                                       p=self.weights.numpy() / torch.sum(self.weights).numpy(),
                                       replace=self.replacement)
        rand_tensor = torch.from_numpy(rand_tensor)
        return iter(rand_tensor.tolist())





#training
input_size = len(data_rm_sparse.columns)
output_size = len(classes)

learning_rate = 0.1
epochs = 3


#reconstraction_optimizer = optim.Adam(reconstraction_model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()
#reconstraction_criterion = nn.MSELoss()



data_train_x=data_rm_sparse
data_train_y=data_rm_sparse.index


#5-fold cross validation
import numpy as np
from sklearn.model_selection import KFold

num_folds = 5

# Generate 5-fold cross-validation indices
kf = KFold(n_splits=num_folds, shuffle=False)
fold_indices = list(kf.split(data))

# Perform 5-fold cross-validation
for fold, (train_indices, test_indices) in enumerate(fold_indices, start=1):

    #define model and optimizer
    model = GO_Net(input_size, output_size,ratio=[0,0,0])
    optimizer = optim.Adam(model.parameters(), lr=0.001)


    # Subset data and annotation based on indices
    x_train = data_train_x.iloc[train_indices].to_numpy()
    y_train = data_train_y[train_indices,]
    
    x_test = data_train_x.iloc[test_indices].to_numpy()
    y_test = data_train_y[test_indices,]

    # Continue with your operations on data_train, anno_train, data_test, and anno_test




    #x_train,x_test,y_train,y_test = ts(data_train_x.to_numpy(),data_train_y.to_numpy(),test_size=0.2,random_state=1, shuffle=True)

    #x_train=x_train[0:400]
    #y_train=y_train[0:400]

    #label_dict={25:0,26:1,27:2,33:3,34:4}
    y_train_relabeled=[label_dict[label] for label in y_train]
    y_test_relabeled=[label_dict[label] for label in y_test]


    #train_size=20000

    #x_train=x_train[0:train_size]
    #y_train_relabeled=y_train_relabeled[0:train_size]

    train_data=MyDataset(x_train,y_train_relabeled)



    #for unbalanced data
    """
    weights=make_weights_for_balanced_classes(train_data,len(classes))
    weights = torch.DoubleTensor(weights)
    sampler = CustomWeightedRandomSampler(weights, len(weights))        #sampler for imbalanced classes
    """

    #train_loader=DataLoader(train_data, batch_size=60, sampler=sampler)
    train_loader=DataLoader(train_data, batch_size=60, shuffle=True)

    num_epochs=15
    # 训练模型
    for epoch in range(num_epochs):
        running_loss = 0.0
        reconstraction_running_loss = 0.0

        for i, batch in enumerate(train_loader, 0):
            inputs, labels = batch
            #print(labels)
            inputs=Variable(inputs).to(torch.float32)
            labels=Variable(labels).to(torch.long)
            # 将梯度缓存清零
            optimizer.zero_grad()

            # 前向传播、计算损失和反向传播
            outputs,_,_,_ = model(inputs)

            loss = criterion(outputs, labels)

            #reconstraction_input=reconstraction_model(outputs)
            #reconstraction_loss = reconstraction_criterion(reconstraction_input, inputs)

            #reconstraction_optimizer.zero_grad()

            #combined_loss=loss+reconstraction_loss
            #combined_loss.backward()
            loss.backward()
        
            optimizer.step()
            #reconstraction_optimizer.step()


            #reconstraction_running_loss += reconstraction_loss.item()
            running_loss += loss.item()

            if i % 40 == 0:
                pass
                #print(i)
                #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            
            if i>400:
                break

        test_data=MyDataset(x_test,y_test_relabeled)
        test_loader=DataLoader(test_data, batch_size=32, shuffle=False)
        result=[]
        for i, batch in enumerate(test_loader):
            inputs, labels = batch
            
            inputs=Variable(inputs).to(torch.float32)

            labels=Variable(labels).to(torch.long)
            
            outputs,_,_,_ = model(inputs)
            pred = list(torch.max(outputs, 1)[1].numpy())
            result.extend(pred)
            #print(pred,labels)
            if i>100:
                break
        accuracy = accuracy_score(y_test_relabeled[0:len(result)],result)

                #########
        print("fold %s-%s" %(fold,epoch),"\taccuracy:\t",accuracy,"\tloss:\t",running_loss / len(train_loader))
        
        #save model
        #pickle.dump(model,open("model/GO_heart.model","wb"))
        
        


fold 1-0 	accuracy:	 0.8904677323860273 	loss:	 0.9541652389859732
fold 1-1 	accuracy:	 0.9034931912374186 	loss:	 0.14886066390086064
fold 1-2 	accuracy:	 0.9123741859088218 	loss:	 0.04005538725780439
fold 1-3 	accuracy:	 0.9230313795145056 	loss:	 0.010275682096526158
fold 1-4 	accuracy:	 0.9248075784487862 	loss:	 0.0036640234860237193


KeyboardInterrupt: 

In [31]:
#gene contribution evaluation for each class weights
GO_weights=model.fc2.weight.detach().numpy().T        #transpose. The weights in torch is transposed by default.
gene_weights=model.fc1.weight.detach().numpy().T        #transpose. The weights in torch is transposed by default.

masked_gene_weights=gene_weights*GO_mask

top_genes_indices=[]

for col in [16]:

    celltype_4_weights=GO_weights[:,col]
    print(classes[col])
    #celltype_4_weights=[abs(weight) for weight in celltype_4_weights]
    
    
    top_indices = np.argsort(celltype_4_weights)[::-1][:100]    #top 40 indices 
    

    gene_count=0
    for index in top_indices:

        GO_term=GO_list[index]

        if GO_type_dict[GO_term]=="P":
            print(GO_term,celltype_4_weights[index])


Hepatocyte_2
GO:1900182 0.077733405
GO:0060041 0.06973743
GO:0007528 0.06663963
GO:0120163 0.06584333
GO:0048286 0.06518815
GO:0046834 0.06464771
GO:0030168 0.06260696
GO:0030514 0.061650097
GO:0035774 0.059527233
GO:0007628 0.057913974
GO:0043408 0.057329282
GO:0007517 0.056171954
GO:0030206 0.054399062
GO:0060828 0.05364213
GO:0060261 0.052877374
GO:0006383 0.051947597
GO:1905606 0.05174742
GO:0040018 0.05145262
GO:0051497 0.05110444
GO:0046777 0.049910963
GO:0051091 0.049896605
GO:0042127 0.049639024
GO:0043524 0.049281046
GO:0000723 0.04873279
GO:0035914 0.04813815
GO:0070588 0.04805571
GO:0006281 0.04804441
GO:0001916 0.04786204
GO:0021915 0.047420464
GO:0099171 0.047342934
GO:0032691 0.04677322
GO:0016485 0.046621487
GO:0007616 0.046363324
GO:0031146 0.0462815
GO:0030198 0.04566292
GO:0045746 0.045563787
GO:0006631 0.045457598
GO:0008283 0.04512153
GO:0090263 0.04510569
GO:0002062 0.044879522
GO:0051726 0.044817712
GO:0006730 0.04476095
GO:0019369 0.044508986
GO:0045786 0.0439017

In [32]:
gene_weights.shape

(5946, 1946)

In [20]:
GO_type_dict={}
with open("human/goa_human.gaf") as f:
    for line in f:
        if line[0]=="!":
            continue

        GO_term=line.split("\t")[4]
        GO_type=line.split("\t")[8]
        if GO_term not in GO_type_dict:
            GO_type_dict[GO_term]=GO_type


In [28]:
classes

['Central_venous_LSECs',
 'Cholangiocytes',
 'Non-inflammatory_Macrophage',
 'alpha-beta_T_Cells',
 'Inflammatory_Macrophage',
 'NK-like_Cells',
 'gamma-delta_T_Cells_1',
 'Hepatocyte_5',
 'Portal_endothelial_Cells',
 'gamma-delta_T_Cells_2',
 'Periportal_LSECs',
 'Hepatocyte_6',
 'Mature_B_Cells',
 'Hepatic_Stellate_Cells',
 'Plasma_Cells',
 'Erythroid_Cells',
 'Hepatocyte_2',
 'Hepatocyte_3',
 'Hepatocyte_1',
 'Hepatocyte_4']